In [1]:
from tensorflow.keras.models import Model

In [2]:
from tensorflow.keras.layers import (
    Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, AvgPool1D, AvgPool2D, Reshape,
    Input, Activation, BatchNormalization, Dense, Add, Lambda, Dropout, LayerNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping

import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
import numpy as np
import pandas as pd
import pickle

In [4]:
import matplotlib.pyplot as plt

In [5]:
min_idx_y = 71
max_idx_y = 150
num_classes_y = max_idx_y - min_idx_y + 1

In [6]:
def crps(y_true, y_pred):
    loss = K.mean(K.sum((K.cumsum(y_pred, axis = 1) - K.cumsum(y_true, axis=1))**2, axis=1))/199
    return loss

In [7]:
def get_conv_net(num_classes_y):
    #_, x, y, z = train_x.shape
    inputdense_players = Input(shape=(11,10,10), name = "playersfeatures_input")
    
    X = Conv2D(128, kernel_size=(1,1), strides=(1,1), activation='relu')(inputdense_players)
    X = Conv2D(160, kernel_size=(1,1), strides=(1,1), activation='relu')(X)
    X = Conv2D(128, kernel_size=(1,1), strides=(1,1), activation='relu')(X)
    
    # The second block of convolutions learns the necessary information per defense player before the aggregation.
    # For this reason the pool_size should be (1, 10). If you want to learn per off player the pool_size must be 
    # (11, 1)
    Xmax = MaxPooling2D(pool_size=(1,10))(X)
    Xmax = Lambda(lambda x1 : x1*0.3)(Xmax)

    Xavg = AvgPool2D(pool_size=(1,10))(X)
    Xavg = Lambda(lambda x1 : x1*0.7)(Xavg)

    X = Add()([Xmax, Xavg])
    X = Lambda(lambda y : K.squeeze(y,2))(X)
    X = BatchNormalization()(X)
    
    X = Conv1D(160, kernel_size=1, strides=1, activation='relu')(X)
    X = BatchNormalization()(X)
    X = Conv1D(96, kernel_size=1, strides=1, activation='relu')(X)
    X = BatchNormalization()(X)
    X = Conv1D(96, kernel_size=1, strides=1, activation='relu')(X)
    X = BatchNormalization()(X)
    
    Xmax = MaxPooling1D(pool_size=11)(X)
    Xmax = Lambda(lambda x1 : x1*0.3)(Xmax)

    Xavg = AvgPool1D(pool_size=11)(X)
    Xavg = Lambda(lambda x1 : x1*0.7)(Xavg)

    X = Add()([Xmax, Xavg])
    X = Lambda(lambda y : K.squeeze(y,1))(X)
    
    X = Dense(96, activation="relu")(X)
    X = BatchNormalization()(X)

    X = Dense(256, activation="relu")(X)
    X = LayerNormalization()(X)
    X = Dropout(0.3)(X)

    outsoft = Dense(num_classes_y, activation='softmax', name = "output")(X)

    model = Model(inputs = [inputdense_players], outputs = outsoft)
    return model

In [19]:
def run_model(X_tensor, y_tensor, n_epochs, batch_size, lr, plot=False, data=''):
    model = get_conv_net(num_classes_y)

    opt = Adam(learning_rate=lr)
    model.compile(loss=crps,
                  optimizer=opt)

    history = model.fit(X_tensor,
                      y_tensor, 
                      epochs=n_epochs,
                      batch_size=batch_size,
                      verbose=1,
                      validation_split=0.2)
    if plot:
        plot_loss(history, batch_size, lr, n_epochs, data)
        
    return model


In [20]:
def plot_loss(history, batch_size, lr, n_epochs, data=''):
    plt.title(f"Learning Curve Batch Size {batch_size} Learning Rate {lr}")
    plt.xlabel('Epoch')
    plt.ylabel('CRPS Loss')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.legend()
    plt.savefig(f"model_plots/plot_bs_{batch_size}_lr_{lr}_ep_{n_epochs}_{data}.pdf")
    plt.show()
    

In [30]:
with open('input_tensors/X_tensor_comb__train.data', 'rb') as f:
    X_train_tensor = pickle.load(f)
with open('input_tensors/y_tensor_comb__train.data', 'rb') as f:
    y_train_tensor = pickle.load(f).astype('float32')
print(X_train_tensor.shape)
print(y_train_tensor.shape)

(4520, 11, 10, 10)
(4520, 80)


In [ ]:
model = run_model(X_train_tensor, y_train_tensor, 300, 2, 1e-5, plot=True, data='comb')

Epoch 1/300
1808/1808 [==============================] - 26s 12ms/step - loss: 0.0419 - val_loss: 0.0391
Epoch 2/300
1808/1808 [==============================] - 20s 11ms/step - loss: 0.0362 - val_loss: 0.0378
Epoch 3/300
1808/1808 [==============================] - 22s 12ms/step - loss: 0.0331 - val_loss: 0.0369
Epoch 4/300
1808/1808 [==============================] - 20s 11ms/step - loss: 0.0315 - val_loss: 0.0349
Epoch 5/300
1808/1808 [==============================] - 20s 11ms/step - loss: 0.0306 - val_loss: 0.0339
Epoch 6/300
1808/1808 [==============================] - 23s 13ms/step - loss: 0.0296 - val_loss: 0.0334
Epoch 7/300
1808/1808 [==============================] - 18s 10ms/step - loss: 0.0287 - val_loss: 0.0331
Epoch 8/300
1808/1808 [==============================] - 18s 10ms/step - loss: 0.0282 - val_loss: 0.0323
Epoch 9/300
1808/1808 [==============================] - 18s 10ms/step - loss: 0.0276 - val_loss: 0.0327
Epoch 10/300
1808/1808 [==============================]

In [32]:
with open('input_tensors/X_tensor_comb_test.data', 'rb') as f:
    X_test_tensor = pickle.load(f)
with open('input_tensors/y_tensor_comb_test.data', 'rb') as f:
    y_test_tensor = pickle.load(f).astype('float32')

In [33]:
model.evaluate(X_test_tensor, y_test_tensor, batch_size=1)

503/503 [==============================] - 2s 3ms/step - loss: 0.0235


0.023463591933250427